In [1]:
import pandas as pd
import re
import os

RESULTS_DIR = './../datasets_reduced/revision/results/'

RESULTS_CHAABEN = 'results_iso_check_with_info_baseline_powerset_withanchor_likepaperbutwithoutanchors.csv'
RESULTS_RAMC = 'results_with_comparable_info.csv' 
RESULTS_RANDOM_RAC = 'dataset_random_baseline_with_completions.csv'
OUTPUT_FILE = 'benchmark_results.csv' #'results_with_comparable_info.csv'

# cd with jupyter into the directory with the results
os.chdir(RESULTS_DIR)


DATASET_RESULTS_FILES = [RESULTS_RANDOM_RAC, RESULTS_RAMC, RESULTS_CHAABEN]
dataset_name = ['random', 'ramc', 'chaaben']


# Model Completion Benchmark
## Some words on the comparison with Chaaben et al.
As elaborated on in the paper, benchmarking the task of model completion is sophisticated. Some approaches perform their evaluation using concepts that are not specific to the task of model completion but rather to the approach.
Some specifically take aspects of a particular domain into account and do not allow to benchmark against (e.g., because also the dataset is not published and their approach can not be applied to other domains).
We investigated an enormous amount of time to understand these difficulties. Eventually, and as advised by the reviewers, we decided to benchmark our approach (and the LLM based approach with random retrieval) with Chaaben et al.'s approach.
Their approach clearly is the closest approach to ours. Anyway, we still had to make several design decisions to be able to make this comparison. These are:

- On their data or on our data? => We applied the approach on our data, since we did not find exactly on which modelset examples their approach has been evaluated. Furthermore, Modelset does not include histories/revisions.
- On which domain models? => We think that a proper level where we can compare several approaches to model completion would be to move to a meta-meta level (following the MOF). This allows us to compare Chaaben et al.'s approach that works for a subset of UML class diagrams and activity diagrams against our approach that works directly on the abstract syntac graph of the models. By interpreting the abstract syntax of our models as class diagrams, we were able to compare (part of) their approach to ours. Having said this: We definitely agree, that domain-specific improvements are necessary and meaningful. Adapting an approach to a specific domain always makes sense. Anyway, for the concrete use case of model completion, a domain-agnostic comparison of approaches should be a goal of the community. For a domain-specific approach it is then the duty of this approach to show improvement over the domain-agnostic baseline, not the otherway round.
In our concrete case, we performed the comparison on our sample of the Revision dataset. This implies that we could not compare against the Activity Diagram implementation. In our comparison, we did focus on concept recommendation and association recommendation. Attribute recommendation in class diagrams is quite comparable to concept recommendation. Indeed, in ECore deciding between a reference to another concept or an attribute of an EClass is more like a design choice and both are considered to be a EStructuralFeature.
- How to implement/adapt their approach? => We looked into the code of the paper and also reused some of their implementation. Some implementation details in the current version of their code were not consistent to the description in the paper. 
We decided to align more to the description in the paper, while making the necessary adaption to work on our code. 
- How to map classes in a class diagram to the Ecore meta-level? => Revision is an Ecore dataset. In our dataset, we had already classified the changes and the changes that are of interest for the recommendation of new concepts, corresponded to a class of changes we called *Add_node* in our samples. We selected these changes (which left us with 51 samples for the revision dataset, which might seem a small number. Anyway, we performed significance test for the results and got significant results for all categories compared. Even without significance tests, it without doubt which approach is leading for the concept recommendation, at least, for our real-world dataset) and mapped the examples we observed to corresponding concept recommendations.
E.g., when an EClass with name *User* is added via a containment to an EClass with the name *Software*, we used the tuple [EClass.Software, EClass.User] in Chaaben et al.'s approach. Depeding on the concrete ECore concept, we replaced *name* by the corresponding identifier (e.g., *key*).
- How to sample context? => In their implementation, the authors sampled (random or all) pairs of concepts from the model at hand. Sampling all pairs is infeasible for real-world models (number of concepts squared concept pairs), and most of them would not be highly related. Since we had simple change graphs at hand, we decided to sample the edges/associations from these simple change graphs.
- Which few-shot examples to choose? => This was one of the hardest choices. Basically this question can be considered even the major difference between the approaches. We could have decided to choose the few-shot samples from the dataset, similar to our approach. Anyway, since we consider this a crucial difference of the approach, we used the few-shot samples exactly as in the implementation of their approach. The few-shot samples were loaded from a file that we used in the re-implementation of their approach.
- How to choose the recommendation? => First of all, it is noteworthy that several recommender systems make a list of k recommendations. We did intentionally decide to set k equal to one, since in a real world scenario (compare to GitHub Copilot), one would typically not come up with a list, but directly integrate one recommendation in the IDE. Anyway, we are also open to consider k>1 in future benchmarks if the community decides that this is necessary. In this case, in our evaluation (similar to Chaaben et al.'s work), we only considered the highes ranked concept. Using the approach as described and implemented by Chaaben et al. this would typicall recommend a concept that is already present in the model. Of course, we excluded these already present concepts and only ranked the list with new concepts and recommended the highest ranked concepet.
- What language model to use? => We used exactly the same language model that has been used in the original implementation (text-davinci-002). We thought about re-implementing their appraoch on the basis of GPT-4. Anyway, this would lead to other necessary design choices. We used their prompts with GPT-4, and, as one would expect, the models starts generating natural language text not related to the concrete use case. This is because text-davinci-002 was not a *chat-like* model, while ChatGPT and GPT-4 are. In consequence, this would require a redsign of the prompts. 
- What metrics to use in the comparison? => To get a clearer picture of the pros and cons of the approaches, we decided to report independently the accuracy of the correct concepts being recommended, the accuracy of correct association being recommended, and we further split in correct type (e.g., EClass in the example above) and correct name (e.g., Software, or User in the example above).


We highly acknowledge and appreciate the work of Chaaben et al. We don't believe that it was intended by the authors to be treated as a baseline for a model completion approach, as indicated by their paper naming *Towards...*. 
Anyway, we were required to deliver a baseline, and Chaaben et al.'s approach was the only one were still a glimpse of feasibility of a comparison could be recognized by the authors of this work.
The differences of the approaches are the language model used (GPT-4 in our case and text-davinci-002 in theirs), the strategy of how the few-shot samples are retrieved, and the representation of the models, concepts, attributes, and relationships in the context of the prompt. 
We are certain that the most important feature is the RAG with semantic retrieval. As we have described in the paper, we retrieved very similar few-shot samples, which of course, help the language model to perform a completion.
The language model has a smaller influence as we know from other experiment. A chat-like prompt and chat-model would improve the results of Chaaben et al. approach, though the semantic retrieval is more curical to the differences observed in the comparison.
Even less, the concrete representation of model and model changes has an influence on the results of model completion. Anyway, there are several dimensions and not all of them can be investigated in this paper and are therefore left as future research.

To better understand the individual influence of several design choice (or combinations theirof) for model completion, we highly encourage the community (around intelligent modeling assistants and for model completion, in particular) to assure that a common dataset (e.g., on the basis of the revision dataset, which includes histories) is evolved, that we work on a common set of reportings, independent of a concrete application domain (such as Simulink, BPM, SysML, or UML) and approach (e.g., using concepts such as edit operations in the evaluation). 
In the broader field of intelligent modeling assistants, we have to work on a common infrastruction to perform research and which enables us to investigate the pros and cons of the designs and design decisions proposed for the use cases or problem settings in the field.

## Threats to validity
We see a large homogenity in the data (a lot of EStringToStringMapEntries with key documentation)

## Differences between the original approach and our implementation and evaluation
- (-) We do not apply the approach to UML classdiagram
- (+) We apply the approach to Ecore diagrams in a way that is compatible to class diagrams (for add concept, add relationship comparison). Add relationship we evaluate many-fold. First, in Ecore add "EReference" can be interpreted, roughly speaking, to adding associations in class diagrams. There are some occurrences of this in our ``add concept'' evlaution. Furthermore, we can evaluate if the correct edge (source, target nodes, directions, and association name), is correct in scenarios where a Reference is added in Ecore.
- (-) We do not use a large random sample and rank
- (+) We use the simple change graph, wich already focuses on the changing part

# Load Dataset

We load all datasets.

In [6]:
datasets =[pd.read_csv(file) for file in DATASET_RESULTS_FILES]

columns = ['dataset', 'same_class', 'same_name', 'same_concept',  'same_association'] #'same_anchor_node',


We adapt the files a little bit for the unified comparison.

In [7]:
# use id as index for chaaben dataset
datasets[2].set_index('id', inplace=True)

# dataset index given by the following constraint (these are the only ones for which we can perform the comparison with Chaaben et al. )
is_add_object = datasets[1]['completion'].str.contains("'changeType': 'Add', 'type': 'object'")

index = is_add_object[is_add_object].index

# we compute same_concept = same_class==True and same_name==True
for i, dataset in enumerate(datasets):
    dataset['same_concept'] = dataset['same_class'] & dataset['same_name']
    
# same association is just a renaming of 'simplified_correct'
for i, dataset in enumerate(datasets):
    if 'simplified_correct' in dataset.columns and not 'same_association' in dataset.columns:
        dataset['same_association'] = dataset['simplified_correct']

# restrict all datasets to the index
for i, dataset in enumerate(datasets):
    datasets[i] = dataset.loc[index]
    
# only keep the following columns  
keep_columns = ['same_class', 'same_name', 'same_concept', 'same_association']
for i, dataset in enumerate(datasets):
    datasets[i] = dataset[keep_columns]

# Stastical evaluation
We compute the correctness (probability of correct completion) for each class. We furthermore compare statistical significance (p-value), w.r.t. to 'random' via binomial test.

In [8]:
# import binomial test from scipy
from scipy.stats import binom_test

# same class
results_df = pd.DataFrame(columns=columns)

for i, dataset in enumerate(datasets):
    # compute all probabilities
    results_df.loc[i, columns[0]] = dataset_name[i] # set the dataset name
    results_df.loc[i+len(datasets), columns[0]] = dataset_name[i] + "_p" # set the dataset name
    for column in columns[1:]:
        p = dataset[column].sum()/len(dataset)
        results_df.loc[i, column] = p    
        # compute binomial test with random baseline = dataset
        # alternative
        if dataset_name[i] == 'random':
            alternative = 'two-sided' # actually we compare to the random, so this will be p-value = 1.0 anyways
        elif dataset_name[i] == 'ramc':
            alternative = 'greater' # we assume that semantical retrieval outperforms random retrieval
        elif dataset_name[i] == 'chaaben':
            alternative = 'less' # our hypothesis is that our random approach is better because it takes into account more similar examples via retrieval of few-shots
        p_values = binom_test(x=dataset[column].sum(), n=len(dataset), p=datasets[0][column].sum()/len(dataset), alternative=alternative)
        results_df.loc[i+len(datasets), column] = p_values

In [18]:
# print the results_df formatted
latex_table = results_df.to_latex(index=False, float_format="%.40f")

In [19]:
print(latex_table)

\begin{tabular}{lllll}
\toprule
dataset & same_class & same_name & same_concept & same_association \\
\midrule
random & 0.7843137254901960675468330919102299958467 & 0.8039215686274510108688673426513560116291 & 0.7647058823529411242247988411691039800644 & 0.6862745098039215729812667632359080016613 \\
random_p & 1.0000000000000000000000000000000000000000 & 1.0000000000000000000000000000000000000000 & 1.0000000000000000000000000000000000000000 & 1.0000000000000000000000000000000000000000 \\
ramc & 0.9411764705882352810561997102922759950161 & 0.9607843137254902243782339610334020107985 & 0.9411764705882352810561997102922759950161 & 0.8039215686274510108688673426513560116291 \\
ramc_p & 0.0022647058234236570536945798437500343425 & 0.0013082316428690038846022192231544067909 & 0.0008507790945998238178391015473778224987 & 0.0443310839736497722118890862930129515007 \\
chaaben & 0.2156862745098039324531669080897700041533 & 0.0980392156862745084433541364887787494808 & 0.098039215686274508443354136

In [11]:
# \begin{tabular}{lllll}
# \toprule
# dataset & same_class & same_name & same_concept & same_association \\
# \midrule
# random & 0.784 & 0.804 & 0.765 & 0.686 \\
# random_p & 1.000 & 1.000 & 1.000 & 1.000 \\
# ramc & 0.941 & 0.961 & 0.941 & 0.804 \\
# ramc_p & 0.002 & 0.001 & 0.001 & 0.044 \\
# chaaben & 0.216 & 0.098 & 0.098 & 0.078 \\
# chaaben_p & 0.000 & 0.000 & 0.000 & 0.000 \\
# \bottomrule
# \end{tabular}


# in the latex table above, add two stars if p-value < 0.01, one star if p-value < 0.05 for the corresponding approach, remove the columns with p-values
latex_table_p = """
\\begin{tabular}{lllll}
\\toprule
dataset & same_class & same_name & same_concept &  same_association \\
\\midrule
random & 0.784 & 0.804 & 0.765 & 0.686 \\
ramc & 0.941** & 0.961** & 0.941** & 0.804* \\
chaaben & 0.216** & 0.098** & 0.098** & 0.078**
\\bottomrule
\\end{tabular}
"""
print(latex_table_p)



\begin{tabular}{lllll}
\toprule
dataset & same_class & same_name & same_concept &  same_association \
\midrule
random & 0.784 & 0.804 & 0.765 & 0.686 \
ramc & 0.941** & 0.961** & 0.941** & 0.804* \
chaaben & 0.216** & 0.098** & 0.098** & 0.078**
\bottomrule
\end{tabular}

